In [1]:
from tensorflow import keras
import pandas as pd
import numpy as np

def generator():
    return keras.preprocessing.image.ImageDataGenerator(
            validation_split=0.25,
            rescale=1./255.,
#            horizontal_flip=True,
#            vertical_flip=True,
#            width_shift_range=0.2,
#            height_shift_range=0.2,
#            rotation_range=5,
    )

def load_subset(path, subset):
    return generator().flow_from_dataframe(
        pd.read_csv(path + '/labels.csv'),
        directory=path + '/final_files',
        x_col='file_name',
        y_col='real_age',
        target_size=(150, 150),
        class_mode='raw',
        subset=subset,
        batch_size=16,
        seed=42
)

def load_train(path):
	return load_subset(path, subset='training')

def load_test(path):
	return load_subset(path, subset='validation')

def create_model(input_shape, weights='/datasets/keras_models/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'):
    backbone = keras.applications.resnet.ResNet50(
        input_shape=input_shape,
        include_top=False,
        weights=weights
    )
    backbone.trainable = True
    model = keras.models.Sequential()
    model.add(backbone)
    model.add(keras.layers.GlobalAveragePooling2D())
    model.add(keras.layers.Dense(512, activation='relu'))
    model.add(keras.layers.Dense(256, activation='relu'))
    model.add(keras.layers.Dense(1, activation='relu'))
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
        loss='mean_squared_error', metrics=['mae'])
    return model

def train_model(model, train_data, test_data, batch_size=None, epochs=5, steps_per_epoch=None, validation_steps=None):
    model.fit(train_data, 
              validation_data=test_data,
              epochs=epochs,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2, shuffle=True)
    return model

2025-09-21 09:15:11.126321: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-21 09:15:11.177283: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-21 09:15:12.445964: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [2]:
train_flow = load_train('./datasets/faces')
test_flow = load_test('./datasets/faces')

Found 5694 validated image filenames.
Found 1897 validated image filenames.


In [3]:
features, target = next(train_flow)
len(train_flow)

356

In [4]:
print('input shape:', features[0].shape)
model = create_model(features[0].shape, './datasets/keras_models/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')
model = train_model(model,
	train_data=train_flow,
	test_data=test_flow)

input shape: (150, 150, 3)


2025-09-21 09:15:13.330553: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/5


/home/wtom/code/notebooks/practicum/vision/.venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


356/356 - 869s - 2s/step - loss: 214.7767 - mae: 10.8180 - val_loss: 547.0052 - val_mae: 18.3229
Epoch 2/5
356/356 - 759s - 2s/step - loss: 77.6261 - mae: 6.8130 - val_loss: 156.7555 - val_mae: 9.4804
Epoch 3/5
356/356 - 750s - 2s/step - loss: 43.9212 - mae: 5.0905 - val_loss: 90.4424 - val_mae: 7.2246
Epoch 4/5
356/356 - 750s - 2s/step - loss: 31.6730 - mae: 4.2948 - val_loss: 94.0120 - val_mae: 7.5466
Epoch 5/5
356/356 - 759s - 2s/step - loss: 24.3900 - mae: 3.7827 - val_loss: 82.9802 - val_mae: 6.9554
